# 📝<span style="font-family:cursive;">Final Part From TPU Session</span>

## 🏆 <a href='https://www.kaggle.com/fauzanalfariz/ranzcr-detection-with-xception'> [Part 1] -> [RANZCR]: Detection With Xception | TPU Session</a>

In [ ]:
# Importing useful libraries

import pandas as pd 
import numpy as np
import os
import tensorflow as tf

import math
import seaborn as sns
import albumentations as A 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os, gc, cv2, random, warnings, math, sys, json, pprint
from glob import glob
from pylab import rcParams

# sklearn
from sklearn.model_selection import  GroupKFold
from sklearn.metrics import roc_auc_score


#import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K

# torch
import torch
import torchvision
from torchvision import transforms

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [ ]:
from kaggle_datasets import KaggleDatasets


data_dir ='../input/ranzcr-clip-catheter-line-classification/'

train = os.path.join(data_dir, 'train.csv')
train_df = pd.read_csv(train)  

sub = os.path.join(data_dir,'sample_submission.csv')
sub_df = pd.read_csv(sub)

test_images = data_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

In [ ]:
def NeedleAugmentation(image, n_needles=2, dark_needles=False, p=0.5, needle_folder='../input/xray-needle-augmentation'):
    
    '''
      Open CV - Based Custom Augmentation
    '''
    
    aug_prob = random.random()
    if aug_prob < p:
        height, width, _ = image.shape  # target image width and height
        needle_images = [im for im in os.listdir(needle_folder) if 'png' in im]

        for _ in range(1, n_needles):
            needle = cv2.cvtColor(cv2.imread(os.path.join(needle_folder, random.choice(needle_images))), cv2.COLOR_BGR2RGB)
            needle = cv2.flip(needle, random.choice([-1, 0, 1]))
            needle = cv2.rotate(needle, random.choice([0, 1, 2]))

            h_height, h_width, _ = needle.shape  # needle image width and height
            roi_ho = random.randint(0, abs(image.shape[0] - needle.shape[0]))
            roi_wo = random.randint(0, abs(image.shape[1] - needle.shape[1]))
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(needle, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of needle in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_needles:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                needle_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                needle_fg = cv2.bitwise_and(needle, needle, mask=mask)

            # Put needle in ROI and modify the target image
            dst = cv2.add(img_bg, needle_fg, dtype=cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

    return image

In [ ]:
TARGET_SIZE = 750

In [ ]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE, TARGET_SIZE), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = NeedleAugmentation(img, n_needles=2, dark_needles=False, p=0.5)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * 8

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model('../input/ranzcr-detection-with-xception/Xception_750_TPU.h5')
                                       #custom_objects={'FixedDropout': tf.keras.layers.Dropout})


In [ ]:
model.summary()

In [ ]:
#test_decoder = build_decoder(with_labels=False, target_size=(TARGET_SIZE, TARGET_SIZE))

In [ ]:
test_df = build_dataset(
    test_images, bsize=batch_size, repeat=False, 
    shuffle=False, augment=False, cache=False,
    #decode_fn=test_decoder 
)

In [ ]:
y_preds = model.predict(test_df, verbose=1)

In [ ]:
sub_df.iloc[:, 1:] = y_preds
display(sub_df)

In [ ]:
sub_df.to_csv('submission.csv',index=False)

# 😄<span style="font-family:cursive;">if this notebook,it is very useful for you let's to upvote :)</span>